# AWS Identity Center Identity Provider Change

Detects a change in the AWS Identity Center (FKA AWS SSO) identity provider.
A change in identity provider allows an attacker to establish persistent access or escalate privileges via user impersonation.


References:
* https://docs.aws.amazon.com/singlesignon/latest/userguide/app-enablement.html
* https://docs.aws.amazon.com/singlesignon/latest/userguide/sso-info-in-cloudtrail.html
* https://docs.aws.amazon.com/service-authorization/latest/reference/list_awsiamidentitycentersuccessortoawssinglesign-on.html

False Positives:
* Authorized changes to the AWS account's identity provider

## Detection Query
Execute the following query to find alerts related to this rule.

In [ ]:
%ingest.source_type="aws:cloudtrail"
eventSource=("sso-directory.amazonaws.com" "sso.amazonaws.com")
eventName=(
AssociateDirectory
DisableExternalIdPConfigurationForDirectory
DisassociateDirectory
EnableExternalIdPConfigurationForDirectory
)
| stats
min(@scnr.datetime) as firstTime,
max(@scnr.datetime) as lastTime,
count() as eventCount
by
userIdentity.arn,
eventSource,
eventName,
awsRegion

## Investigation Steps
1. **Identify the Principal**: Who performed the action?
2. **Review Context**: What else did this principal do?
3. **Check Permissions**: Did the principal have legitimate reasons for this action?
4. **Verify Resource**: Is the resource critical or sensitive?

In [ ]:
%ingest.source_type:aws:cloudtrail
userIdentity.arn: <PRINCIPAL_ARN>
| count(eventName) by eventName